# 01 构建拼音-汉字字典与统计占位

本 Notebook 读取 `Chara.gb` 与 `TONEPY.txt`，演示：
1. 解析两类来源的映射格式
2. 统一为 `pinyin -> [chars]` 与 `char -> [pinyin_with_tone]` 两种索引结构
3. 去重、排序（按字符频次或出现顺序；当前暂用原始顺序）
4. 为后续 HMM 构建发射概率提供原始候选集合

后续可扩展：结合语料统计每个 (char, pinyin) 频次，转成发射概率矩阵。当前仅做结构演示。

In [4]:
from pathlib import Path
import re
from collections import defaultdict

DATA_DIR = Path('../usrs')  # 以仓库根相对定位
chara_path = DATA_DIR / 'Chara.gb'
tonepy_path = DATA_DIR / 'TONEPY.txt'

print('Chara.gb exists:', chara_path.exists())
print('TONEPY.txt exists:', tonepy_path.exists())

Chara.gb exists: True
TONEPY.txt exists: True


## 解析 Chara.gb
格式示例：`啊 a1 a2 a4 a5` -> 第一个是汉字，后面是带声调数字的拼音候选。
注意：有些行包含全角字符或多义条目，需要 strip。空行与异常行需跳过。

In [5]:
def parse_chara_gb(path: Path):
    char_to_pinyins = {}  # char -> list[pinyinTone]
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            ch = parts[0]  # 单字
            # 后面是若干声调拼音，如 a1 a2 a4 a5
            pys = parts[1:]
            char_to_pinyins[ch] = pys
    return char_to_pinyins

char_to_pinyins = parse_chara_gb(chara_path)
list(char_to_pinyins.items())[:10]

[('啰', ['luo1', 'luo5']),
 ('潾', ['lin2']),
 ('Ｑ', ['qiu1']),
 ('啊', ['a1', 'a2', 'a4', 'a5']),
 ('阿', ['a1', 'e1', 'a4']),
 ('埃', ['ai1']),
 ('挨', ['ai1', 'ai2', 'ai5']),
 ('哎', ['ai1']),
 ('唉', ['ai1', 'ai4']),
 ('哀', ['ai1'])]

## 解析 TONEPY.txt
格式示例：`ai4	爱碍艾...` -> 第一列为带声调拼音，第二列为连续汉字字符串。
空列或无汉字时需记录空列表，便于后续一致化。

In [6]:
def parse_tonepy(path: Path):
    pinyin_to_chars = {}  # pinyinTone -> list[char]
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            # 去除换行符（兼容 \n / \r\n）
            line = line.rstrip('\r\n')
            if not line:
                continue
            if '	' not in line:  # 需包含制表符分隔
                continue
            py_tone, chars = line.split('	', 1)
            chars = chars.strip()
            # 将连续字符串展开为单字列表
            pinyin_to_chars[py_tone] = list(chars) if chars else []
    return pinyin_to_chars

pinyin_to_chars = parse_tonepy(tonepy_path)
list(pinyin_to_chars.items())[:8]

[('a1', ['阿', '啊', '呵', '腌', '吖', '锕']),
 ('a2', ['啊', '呵', '嗄']),
 ('a3', ['啊', '呵']),
 ('a4', ['啊', '呵']),
 ('a5', ['阿', '啊', '呵']),
 ('ai1', ['哀', '挨', '埃', '唉', '哎', '捱', '锿']),
 ('ai2', ['呆', '挨', '癌', '皑', '捱']),
 ('ai3', ['矮', '哎', '蔼', '霭', '嗳'])]

## 构建无声调拼音索引
HMM 发射概率常用无声调形式（用户一般输入不含声调）。需要：
1. 去除末尾数字 (1-5) 形成 base pinyin
2. 合并同 base pinyin 下所有汉字去重
3. 可按频次排序（当前无频次，维持解析顺序）

In [7]:
digit_re = re.compile(r'([a-z]+)[1-5]$')

def tone_to_base(py_tone: str) -> str:
    m = digit_re.match(py_tone)
    return m.group(1) if m else py_tone

base_pinyin_to_chars = defaultdict(list)
for py_tone, chars in pinyin_to_chars.items():
    base = tone_to_base(py_tone)
    for ch in chars:
        if ch not in base_pinyin_to_chars[base]:  # 保持顺序去重
            base_pinyin_to_chars[base].append(ch)

# 示例如 'ai'
base_pinyin_to_chars['ai'][:15]

['哀', '挨', '埃', '唉', '哎', '捱', '锿', '呆', '癌', '皑', '矮', '蔼', '霭', '嗳', '爱']

## 结合 char->pinyins 反向补充
有些字可能只在 `Chara.gb` 中出现，补充其拼音到 base 映射里。

In [8]:
for ch, py_tones in char_to_pinyins.items():
    for py_tone in py_tones:
        base = tone_to_base(py_tone)
        bucket = base_pinyin_to_chars[base]
        if ch not in bucket:
            bucket.append(ch)
len(base_pinyin_to_chars), base_pinyin_to_chars['a'][:20]

(420, ['阿', '啊', '呵', '腌', '吖', '锕', '嗄'])

## 保存结果（示例 JSON）
后续 HMM 发射概率将基于 (char, pinyin) 频次；当前仅导出候选集合。
实际项目中应放入 `resources/pinyin_map.json`（不含声调）。

In [ ]:
import json, os
resources_dir = Path('..') / 'resources'
resources_dir.mkdir(exist_ok=True)
output_path = resources_dir / 'pinyin_map.example.json'
with output_path.open('w', encoding='utf-8') as f:
    json.dump(base_pinyin_to_chars, f, ensure_ascii=False, indent=2)
output_path, output_path.stat().st_size

(WindowsPath('../resources/pinyin_map.example.json'), 96479)

## 后续步骤建议
1. 引入语料（人民日报等）统计单字与双字频次。
2. 构建初始概率 π（句首字频次归一化）。
3. 构建转移概率 A：bigram 平滑。
4. 构建发射概率 B：估计 P(pinyin|char)=count(char emits pinyin)/count(char)。
5. 实现 Viterbi：输入无声调拼音序列 -> 输出最优汉字序列。
6. 增加 Top-K：改造回溯结构或用 beam search。
7. 评估：字符准确率；可选词切分后词准确率。

（本 Notebook 仅完成基础字典转换示例。）